# Notebook Initialization

In [6]:
# %load nb_init.py

from pathlib import Path
import pandas as pd

base_dir = Path.cwd().parent
config_dir = base_dir / "config"
data_dir = base_dir / "data"
docs_dir = base_dir / "docs"
figures_dir = docs_dir / "figures"
models_dir = base_dir / "models"
logs_dir = base_dir / "logs"
images_input_dir = data_dir / "COVID19"
images_pp_dir = data_dir / "COVID19-preprocessed"
preprocessed_dir = data_dir / "preprocessed"
output_dir = data_dir / "output"

# Directories used to train the CNN (image by image) 
cnn_data_dir = data_dir / "modelling" / "cnn"
cnn_train_dir = cnn_data_dir / "train"
cnn_test_dir = cnn_data_dir / "test"

metadata_file = images_input_dir / "metadata.csv"
labels_file = images_input_dir / "unzip_filenames.csv"
preprocessed_labels_file = preprocessed_dir / "labels.parquet"

feature_extractor_model_file = models_dir / "feature_extractor.tf"

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

config_file = config_dir / "tfg.conf"

from pyhocon import ConfigFactory
config = None

def load_config():
    return ConfigFactory.parse_file(config_file)

config = load_config()
    
import sys

if str(base_dir / "src") not in sys.path:
    sys.path.append(str(base_dir / "src"))

%load_ext autoreload

%autoreload 2

In [7]:
spark

In [8]:
from tfg import DataRepository

repo = DataRepository(config=config, base_data_path=data_dir)

# Load labels

In [5]:
labels = spark.read.parquet(str(preprocessed_labels_file))

labels.show(5)

+----------+-------+-------+-----+---------+
|patient_id|scan_id|n_slice|label|num_clips|
+----------+-------+-------+-----+---------+
|         0|   3131|    285|   CP|        5|
|         0|   3132|     42|   CP|        1|
|         0|   3133|    290|   CP|        5|
|         0|   3134|     37|   CP|        1|
|         0|   3135|    269|   CP|        4|
+----------+-------+-------+-----+---------+
only showing top 5 rows



In [6]:
num_labels = labels.count()
num_labels

4178

In [7]:
labels\
    .groupBy("label")\
    .agg(
        F.count("*").alias("count"),
        F.round((F.count("*") / num_labels), 4).alias("pct"),
    )\
    .show()

+------+-----+------+
| label|count|   pct|
+------+-----+------+
|    CP| 1556|0.3724|
|   NCP| 1544|0.3696|
|Normal| 1078| 0.258|
+------+-----+------+



In [8]:
labels.select("patient_id").distinct().count()

2742

# Split patient ids into train / test

In [9]:
seed = config.get_int("tfg.seed")
test_fraction = config.get_float("tfg.training.test_fraction")
val_fraction = config.get_float("tfg.training.val_fraction")
train_fraction = 1 - test_fraction - val_fraction
split_weights = [train_fraction, val_fraction, test_fraction]
split_weights

[0.8, 0.1, 0.1]

In [10]:
patient_ids = labels.select("patient_id").distinct()

id_dfs = patient_ids.randomSplit(split_weights, seed=seed)
train_ids = id_dfs[0]
val_ids =  id_dfs[1]
test_ids =  id_dfs[2]

train_labels = labels.join(train_ids, ["patient_id"], "inner")
val_labels = labels.join(val_ids, ["patient_id"], "inner")
test_labels = labels.join(test_ids, ["patient_id"], "inner")

train_count = train_labels.count()
val_count = val_labels.count()
test_count = test_labels.count()

print(f"Training rows: {train_count}")
print(f"Validation rows: {val_count}")
print(f"Test rows: {test_count}")

train_labels.show(5)
val_labels.show(5)
test_labels.show(5)

Training rows: 3350
Validation rows: 402
Test rows: 426
+----------+-------+-------+-----+---------+
|patient_id|scan_id|n_slice|label|num_clips|
+----------+-------+-------+-----+---------+
|         0|   3131|    285|   CP|        5|
|         0|   3132|     42|   CP|        1|
|         0|   3133|    290|   CP|        5|
|         0|   3134|     37|   CP|        1|
|         0|   3135|    269|   CP|        4|
+----------+-------+-------+-----+---------+
only showing top 5 rows

+----------+-------+-------+-----+---------+
|patient_id|scan_id|n_slice|label|num_clips|
+----------+-------+-------+-----+---------+
|      1075|   3118|    553|   CP|        8|
|      1075|   3119|     70|   CP|        1|
|      1080|   3125|     64|   CP|        1|
|      1093|   3311|    173|   CP|        3|
|      1394|   3847|     62|   CP|        1|
+----------+-------+-------+-----+---------+
only showing top 5 rows

+----------+-------+-------+-----+---------+
|patient_id|scan_id|n_slice|label|num_c

In [11]:
print("Train label counts and percentages")
train_labels\
    .groupBy("label")\
    .agg(
        F.count("*").alias("count"),
        F.round((F.count("*") / train_count), 2).alias("pct"),
    )\
    .show()

print("Validation label counts and percentages")
val_labels\
    .groupBy("label")\
    .agg(
        F.count("*").alias("count"),
        F.round((F.count("*") / val_count), 2).alias("pct"),
    )\
    .show()

print("Test label counts and labels")
test_labels\
    .groupBy("label")\
    .agg(
        F.count("*").alias("count"),
        F.round((F.count("*") / test_count), 2).alias("pct"),
    )\
    .show()

Train label counts and percentages
+------+-----+----+
| label|count| pct|
+------+-----+----+
|    CP| 1229|0.37|
|   NCP| 1229|0.37|
|Normal|  892|0.27|
+------+-----+----+

Validation label counts and percentages
+------+-----+----+
| label|count| pct|
+------+-----+----+
|    CP|  152|0.38|
|   NCP|  158|0.39|
|Normal|   92|0.23|
+------+-----+----+

Test label counts and labels
+------+-----+----+
| label|count| pct|
+------+-----+----+
|    CP|  175|0.41|
|   NCP|  157|0.37|
|Normal|   94|0.22|
+------+-----+----+



# Create datasets for the CNN

For each of the train/validation/test sets we have, we will create a pandas dataframe, and we'll use these dataframe to create the data generators for our model.

In [12]:
from dataclasses import dataclass

from pyspark.sql import Row


@dataclass
class Observation:
    patient_id: int
    scan_id: int
    n_slice: int
    num_clips: int
    label: str

    @staticmethod
    def from_row(r: Row) -> "Observation":
        return Observation(
            patient_id=r.patient_id,
            scan_id=r.scan_id,
            n_slice=r.n_slice,
            num_clips=r.num_clips,
            label=r.label,
        )

In [20]:
from tqdm.notebook import tqdm
import pandas as pd
from pathlib import Path

def create_df(config, labels):
    raw_images = {
        "label": [],
        "file": [],
        "patient_id": [],
        "scan_id": [],
        "n_slice": [],
        "num_clips": [],
        "seq_num": [],
        "clip_num": [],
    }

    IMAGES_PER_CLIP = config.get_int("tfg.training.images_per_clip")

    for r in labels.collect():
        obs = Observation.from_row(r)
        scan_path = Path(f"{obs.label}/{obs.patient_id}/{obs.scan_id}")
        absolute_scan_path = images_input_dir / scan_path
        # different directories have different formats
        img_ext = next(absolute_scan_path.glob("*.*")).suffix

        scan_images = []
        seq_nums = []
        clip_nums = []

        seq_num = 0
        images_input_dir_str = str(images_input_dir)
        path_offset = len(images_input_dir_str) + 1
        for seq_num, img in enumerate(absolute_scan_path.glob(f"*{img_ext}")):
            img_to_append = str(img)[path_offset:]
            scan_images.append(img_to_append)
            seq_nums.append(seq_num)
            clip_nums.append(seq_num // IMAGES_PER_CLIP)
            seq_num = seq_num + 1

        raw_images["label"].extend([obs.label] * obs.n_slice)
        raw_images["file"].extend(scan_images)
        raw_images["patient_id"].extend([obs.patient_id] * obs.n_slice)
        raw_images["scan_id"].extend([obs.scan_id] * obs.n_slice)
        raw_images["n_slice"].extend([obs.n_slice] * obs.n_slice)
        raw_images["num_clips"].extend([obs.patient_id] * obs.n_slice)
        raw_images["seq_num"].extend(seq_nums)
        raw_images["clip_num"].extend(clip_nums)
        
    return pd.DataFrame(raw_images)

In [21]:
train_df = create_df(config, train_labels)
print(f"{train_df.shape}")
train_df

(331286, 8)


,label,file,patient_id,scan_id,n_slice,num_clips,seq_num,clip_num
0,CP,CP/0/3131/0275.png,0,3131,285,0,0,0
1,CP,CP/0/3131/0064.png,0,3131,285,0,1,0
2,CP,CP/0/3131/0083.png,0,3131,285,0,2,0
3,CP,CP/0/3131/0160.png,0,3131,285,0,3,0
4,CP,CP/0/3131/0127.png,0,3131,285,0,4,0
...,...,...,...,...,...,...,...,...
331281,Normal,Normal/1924/379/0014.png,1924,379,98,1924,93,1
331282,Normal,Normal/1924/379/0001.png,1924,379,98,1924,94,1
331283,Normal,Normal/1924/379/0027.png,1924,379,98,1924,95,1
331284,Normal,Normal/1924/379/0031.png,1924,379,98,1924,96,1


In [26]:
repo.save("train_df", train_df)

True

In [27]:
val_df = create_df(config, val_labels)
print(f"{val_df.shape}")
val_df

(39828, 8)


,label,file,patient_id,scan_id,n_slice,num_clips,seq_num,clip_num
0,CP,CP/1075/3118/0543.jpg,1075,3118,553,1075,0,0
1,CP,CP/1075/3118/0174.jpg,1075,3118,553,1075,1,0
2,CP,CP/1075/3118/0130.jpg,1075,3118,553,1075,2,0
3,CP,CP/1075/3118/0525.jpg,1075,3118,553,1075,3,0
4,CP,CP/1075/3118/0152.jpg,1075,3118,553,1075,4,0
...,...,...,...,...,...,...,...,...
39823,Normal,Normal/1920/375/0014.png,1920,375,100,1920,95,1
39824,Normal,Normal/1920/375/0001.png,1920,375,100,1920,96,1
39825,Normal,Normal/1920/375/0027.png,1920,375,100,1920,97,1
39826,Normal,Normal/1920/375/0031.png,1920,375,100,1920,98,1


In [28]:
repo.save("val_df", val_df)

True

In [29]:
test_df = create_df(config, test_labels)
print(f"{test_df.shape}")
test_df

(40415, 8)


,label,file,patient_id,scan_id,n_slice,num_clips,seq_num,clip_num
0,CP,CP/1/3143/0275.png,1,3143,300,1,0,0
1,CP,CP/1/3143/0064.png,1,3143,300,1,1,0
2,CP,CP/1/3143/0083.png,1,3143,300,1,2,0
3,CP,CP/1/3143/0160.png,1,3143,300,1,3,0
4,CP,CP/1/3143/0286.png,1,3143,300,1,4,0
...,...,...,...,...,...,...,...,...
40410,Normal,Normal/1917/372/0014.png,1917,372,96,1917,91,1
40411,Normal,Normal/1917/372/0001.png,1917,372,96,1917,92,1
40412,Normal,Normal/1917/372/0027.png,1917,372,96,1917,93,1
40413,Normal,Normal/1917/372/0031.png,1917,372,96,1917,94,1


In [30]:
repo.save("test_df", test_df)

True

# Data Generators

In [9]:
train_df = repo.load("train_df")
train_df.head(5)

,label,file,patient_id,scan_id,n_slice,num_clips,seq_num,clip_num
0,CP,CP/0/3131/0275.png,0,3131,285,0,0,0
1,CP,CP/0/3131/0064.png,0,3131,285,0,1,0
2,CP,CP/0/3131/0083.png,0,3131,285,0,2,0
3,CP,CP/0/3131/0160.png,0,3131,285,0,3,0
4,CP,CP/0/3131/0127.png,0,3131,285,0,4,0


In [10]:
val_df = repo.load("val_df")
val_df.head(5)

,label,file,patient_id,scan_id,n_slice,num_clips,seq_num,clip_num
0,CP,CP/1075/3118/0543.jpg,1075,3118,553,1075,0,0
1,CP,CP/1075/3118/0174.jpg,1075,3118,553,1075,1,0
2,CP,CP/1075/3118/0130.jpg,1075,3118,553,1075,2,0
3,CP,CP/1075/3118/0525.jpg,1075,3118,553,1075,3,0
4,CP,CP/1075/3118/0152.jpg,1075,3118,553,1075,4,0


In [34]:
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import load_img

img_sizes = set()

# We know that all images in a scan have the same size, so we only need 1 image per scan,
# that's why we can remove the duplicatees based on patient id + scan id
for r in tqdm(train_df.drop_duplicates(["patient_id", "scan_id"]).iterrows()):
    img_path = images_input_dir / r[1]["file"]
    img_sizes.add(load_img(img_path).size)

img_sizes

{(512, 512), (768, 768), (1024, 1024), (1168, 1168)}

In [35]:
val_img_sizes = set()

# We know that all images in a scan have the same size, so we only need 1 image per scan,
# that's why we can remove the duplicatees based on patient id + scan id
for r in tqdm(val_df.drop_duplicates(["patient_id", "scan_id"]).iterrows()):
    img_path = images_input_dir / r[1]["file"]
    val_img_sizes.add(load_img(img_path).size)

val_img_sizes

{(512, 512)}